In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

In [2]:
df_birth_and_emerg = pd.read_csv("../../data/df_birth_and_emerg.csv")

In [3]:
reference_df = pd.read_csv("../../data/reference_df.csv")

In [4]:
df_birth_and_emerg

,Unnamed: 0,uniqueID,drugName,condition,review,rating,date,usefulCount
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
2,6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5
3,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10
...,...,...,...,...,...,...,...,...
31246,161268,76151,Portia,Birth Control,"""I switched to Portia 12 days ago when I start...",1,28-Jan-14,9
31247,161269,73058,Ethinyl estradiol / norethindrone,Birth Control,"""When I first starting taking Lo Loestrin Fe, ...",10,20-Apr-15,13
31248,161271,148859,Mirena,Birth Control,"""My experience: Painful insertion but I expect...",10,26-Jan-16,4
31249,161273,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0


In [5]:
reference_df

,Unnamed: 0,brand name,generic name,type
0,0,Nexplanon (Pro),etonogestrel,standard
1,1,Plan B One-Step (Pro),levonorgestrel,standard
2,2,Mirena (Pro),levonorgestrel,standard
3,3,Depo Provera (Pro),medroxyprogesterone,standard
4,4,Kyleena (Pro),levonorgestrel,standard
...,...,...,...,...
268,268,condom,Latex Polyurethane Polyisoprene,Male condom
269,269,Depo-SubQ Provera 104,NaN,Shot/injection
270,270,FemCap,spermicide Silicone,Cervical cap
271,271,Today sponge,spermidice,Sponge


In [6]:
reference_df = reference_df.drop(columns=["Unnamed: 0"])

In [7]:
reference_df

,brand name,generic name,type
0,Nexplanon (Pro),etonogestrel,standard
1,Plan B One-Step (Pro),levonorgestrel,standard
2,Mirena (Pro),levonorgestrel,standard
3,Depo Provera (Pro),medroxyprogesterone,standard
4,Kyleena (Pro),levonorgestrel,standard
...,...,...,...
268,condom,Latex Polyurethane Polyisoprene,Male condom
269,Depo-SubQ Provera 104,NaN,Shot/injection
270,FemCap,spermicide Silicone,Cervical cap
271,Today sponge,spermidice,Sponge


In [8]:
#Creating a function to compare drugName with both brand name and generic name

def find_match(drug_name, ref_df):
    
    #Checking if drug_name is not null
    
    if pd.isna(drug_name):
        return pd.Series([None, None])

    for idx, row in ref_df.iterrows():
        
        #Ensuring that both brand name and generic name are not null
        
        brand_name = row['brand name'] if pd.notna(row['brand name']) else ''
        generic_name = row['generic name'] if pd.notna(row['generic name']) else ''
        
        # Comparing drugName with brand name and generic name with fuzzy
        
        brand_similarity = fuzz.ratio(drug_name, brand_name)
        generic_similarity = fuzz.ratio(drug_name, generic_name)
        
        # If the similarity is within the 60-100% range, return the generic name and type
        
        if 60 <= brand_similarity <= 100 or 60 <= generic_similarity <= 100:
            return pd.Series([row['generic name'], row['type']])
    
    return pd.Series([None, None])

# Step 3: Apply the function to each row in test_df to find matching rows in reference_df
df_birth_and_emerg[['matched_generic_name', 'matched_type']] = df_birth_and_emerg['drugName'].apply(find_match, ref_df=reference_df)

In [9]:
df_birth_and_emerg

,Unnamed: 0,uniqueID,drugName,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,None,None
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,None,None
2,6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,etonogestrel,standard
3,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,ethinyl estradiol / norethindrone,standard
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,etonogestrel,standard
...,...,...,...,...,...,...,...,...,...,...
31246,161268,76151,Portia,Birth Control,"""I switched to Portia 12 days ago when I start...",1,28-Jan-14,9,ethinyl estradiol / levonorgestrel,standard
31247,161269,73058,Ethinyl estradiol / norethindrone,Birth Control,"""When I first starting taking Lo Loestrin Fe, ...",10,20-Apr-15,13,ethinyl estradiol / norethindrone,standard
31248,161271,148859,Mirena,Birth Control,"""My experience: Painful insertion but I expect...",10,26-Jan-16,4,levonorgestrel,standard
31249,161273,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,etonogestrel,standard


In [10]:
df_birth_and_emerg["condition"].value_counts()

condition
Birth Control              28788
Emergency Contraception     2463
Name: count, dtype: int64

In [11]:
df_birth_and_emerg.to_csv("../../data/birth_and_emerg_MATCHED_60_percent.csv")

In [12]:
df_birth_and_emerg.isnull().sum()

Unnamed: 0                 0
uniqueID                   0
drugName                   0
condition                  0
review                     0
rating                     0
date                       0
usefulCount                0
matched_generic_name    1840
matched_type            1840
dtype: int64

In [34]:
df_birth_and_emerg

,Unnamed: 0,uniqueID,drugName,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,None,None
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,None,None
2,6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,etonogestrel,standard
3,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,ethinyl estradiol / norethindrone,standard
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,etonogestrel,standard
...,...,...,...,...,...,...,...,...,...,...
31246,161268,76151,Portia,Birth Control,"""I switched to Portia 12 days ago when I start...",1,28-Jan-14,9,None,None
31247,161269,73058,Ethinyl estradiol / norethindrone,Birth Control,"""When I first starting taking Lo Loestrin Fe, ...",10,20-Apr-15,13,ethinyl estradiol / norethindrone,standard
31248,161271,148859,Mirena,Birth Control,"""My experience: Painful insertion but I expect...",10,26-Jan-16,4,levonorgestrel,Intrauterine devices (IUD)
31249,161273,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,etonogestrel,standard


In [35]:
df_birth_and_emerg[df_birth_and_emerg["matched_type"] == "emergency"]

,Unnamed: 0,uniqueID,drugName,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
40,197,26839,Ulipristal,Emergency Contraception,"""This pill works!\r\r\nI decided to share my e...",10,8-Sep-16,10,ulipristal,emergency
75,383,6471,Ocella,Birth Control,"""I&#039;m 20, 5ft and 97lbs. I&#039;ve been on...",8,2-Feb-13,22,ulipristal,emergency
107,529,6482,Ocella,Birth Control,"""I&#039;ve been taking Ocella for 4 months, an...",9,19-Oct-09,3,ulipristal,emergency
145,731,213061,EContra EZ,Emergency Contraception,"""I had unprotected sex the day before I starte...",10,2-Aug-16,14,levonorgestrel,emergency
298,1554,6425,Ocella,Birth Control,"""I have been on Ocella now for over 2 years. ...",9,2-Jun-11,1,ulipristal,emergency
...,...,...,...,...,...,...,...,...,...,...
30893,159443,105074,Next Choice,Emergency Contraception,"""I took next choice within 24 hours of unprote...",5,18-May-15,9,levonorgestrel,emergency
30934,159640,6427,Ocella,Birth Control,"""I have been taking Ocella now for about 6 mon...",5,20-Apr-11,2,ulipristal,emergency
30971,159868,6389,Ocella,Birth Control,"""Starting in elementary school, my acne was ho...",9,14-Jan-16,4,ulipristal,emergency
31096,160618,26821,Ulipristal,Emergency Contraception,"""Ella gets it done. The side effects are minim...",10,25-Sep-13,18,ulipristal,emergency
